In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import nltk
# from nltk.corpus import stopwords
from gensim.models import KeyedVectors

nltk.download("punkt_tab")
# nltk.download("stopwords")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Steven\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_path = './data/'
# data_path = 'G:/GaTech Dropbox/Nian Liu/CSE6250_BDH/Project/Heart-failure-prediction/data/'
df_data = pd.read_csv(data_path + 'data_processed.csv')
df_data['TOKENS'] = df_data['CLEAN_TEXT'].apply(nltk.word_tokenize)

# stop_words = set(stopwords.words('english'))
# def remove_stopwords_nltk(list_tokens):
#     return [token for token in list_tokens if token.lower() not in stop_words]
# df_data['TOKENS'] = df_data['TOKENS'].apply(remove_stopwords_nltk)

df_data_genRe = df_data[['TOKENS', 'GEN_RE']]
df_data_30Re = df_data[['TOKENS', '30_RE']]

Download word2vec file from Kaggle: https://www.kaggle.com/datasets/alexiscorona/pubmed-and-pmc-w2v/

In [4]:
w2v = KeyedVectors.load_word2vec_format("PubMed-and-PMC-w2v.bin", binary=True)
# w2v = KeyedVectors.load_word2vec_format("PubMed-w2v.bin", binary=True)

In [5]:
pos_genRe_idx = np.where((df_data['GEN_RE'] == 1))[0]
neg_genRe_idx = np.where((df_data['GEN_RE'] == 0))[0]
pos_30Re_idx = np.where((df_data['30_RE'] == 1))[0]
neg_30Re_idx = np.where((df_data['30_RE'] == 0))[0]

In [6]:
def vectorize(tokens):
    vectors = [w2v[token] if token in w2v else np.random.uniform(-1, 1, (200,)).astype(np.float32) for token in tokens]
    return np.array(vectors)

def generate_dataset(df_data, pos_idx, neg_idx, re_type):
    num_pos = len(pos_idx)
    # labels = [1] * num_pos + [0] * num_pos
    neg_idx_sample = np.random.choice(neg_idx, size=num_pos, replace=False)
    all_idx = pos_idx.tolist() + neg_idx_sample.tolist()
    tokens_all = df_data.iloc[all_idx, :]['TOKENS'].to_list()
    labels = df_data.iloc[all_idx][re_type].to_list()
    vectors_all = [vectorize(tokens) for tokens in tokens_all]
    if re_type == '30_RE':
        return vectors_all, labels, neg_idx_sample
    return vectors_all, labels

def generate_dataset_oversample(df_data, pos_idx, neg_idx, re_type):
    num_neg = len(neg_idx)
    # labels = [1] * num_pos + [0] * num_pos
    pos_idx_sample = np.random.choice(pos_idx, size=num_neg, replace=True)
    all_idx = neg_idx.tolist() + pos_idx_sample.tolist()
    tokens_all = df_data.iloc[all_idx, :]['TOKENS'].to_list()
    labels = df_data.iloc[all_idx][re_type].to_list()
    vectors_all = [vectorize(tokens) for tokens in tokens_all]
    return vectors_all, labels

def collate_data(batch):
    batch = sorted(batch, key=lambda x: x[0].shape[0])[::-1]
    max_len = batch[0][0].shape[0]
    labels = [i[1] for i in batch]
    
    vectors = []
    for itm in batch:
        if itm[0].shape[0] < max_len:
            vectors.append(np.pad(itm[0], ((0, max_len - itm[0].shape[0]), (0, 0)), mode='constant', constant_values=0))
        else:
            vectors.append(itm[0])
    vectors = np.stack(vectors, axis=0)
    return torch.tensor(vectors, dtype=torch.float), torch.tensor(labels, dtype=torch.float)

In [7]:
class HFDataset(Dataset):
    def __init__(self, vectors_all, labels):
        self.vectors_all = vectors_all
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        vectors = self.vectors_all[index]
        return vectors, self.labels[index]

In [8]:
class HFCNN(nn.Module):
    def __init__(self, out_channels):
        super(HFCNN, self).__init__()
        self.out_channels = out_channels
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(1, 200))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(2, 200))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(3, 200))
        # self.conv4 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(4, 200))
        # self.conv5 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(5, 200))
        # self.conv6 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(6, 200))
        # self.conv7 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(7, 200))
        # self.conv8 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(8, 200))
        # self.conv9 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(9, 200))
        # self.conv10 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(10, 200))
        # self.conv11 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(11, 200))
        # self.conv12 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(12, 200))
        
        # self.bn1 = nn.BatchNorm1d(out_channels)
        # self.bn2 = nn.BatchNorm1d(out_channels)
        # self.bn3 = nn.BatchNorm1d(out_channels)
        
        self.dp = nn.Dropout(0.5) # default is 0.5, 0.4 is worse, 0.6 gets lower val loss but acc seems to be similar
        self.FC1 = nn.Linear(in_features=out_channels * 3, out_features=1)
        # self.FC2 = nn.Linear(in_features=16, out_features=1)
        
    def forward(self, x):
        x = x.unsqueeze(1)
                
        feature1 = F.relu(self.conv1(x).squeeze(3))
        feature2 = F.relu(self.conv2(x).squeeze(3))
        feature3 = F.relu(self.conv3(x).squeeze(3))
        # feature4 = F.relu(self.conv4(x).squeeze(3))
        # feature5 = F.relu(self.conv5(x).squeeze(3))
        # feature6 = F.relu(self.conv6(x).squeeze(3))
        # feature7 = F.relu(self.conv7(x).squeeze(3))
        # feature8 = F.relu(self.conv8(x).squeeze(3))
        # feature9 = F.relu(self.conv9(x).squeeze(3))
        # feature10 = F.relu(self.conv10(x).squeeze(3))
        # feature11 = F.relu(self.conv11(x).squeeze(3))
        # feature12 = F.relu(self.conv12(x).squeeze(3))
                
        feature1 = F.max_pool1d(feature1, feature1.size(2)).squeeze(2)
        feature2 = F.max_pool1d(feature2, feature2.size(2)).squeeze(2)
        feature3 = F.max_pool1d(feature3, feature3.size(2)).squeeze(2)
        # feature4 = F.max_pool1d(feature4, feature4.size(2)).squeeze(2)
        # feature5 = F.max_pool1d(feature5, feature5.size(2)).squeeze(2)
        # feature6 = F.max_pool1d(feature6, feature6.size(2)).squeeze(2)
        # feature7 = F.max_pool1d(feature7, feature7.size(2)).squeeze(2)
        # feature8 = F.max_pool1d(feature8, feature8.size(2)).squeeze(2)
        # feature9 = F.max_pool1d(feature9, feature9.size(2)).squeeze(2)
        # feature10 = F.max_pool1d(feature10, feature10.size(2)).squeeze(2)
        # feature11 = F.max_pool1d(feature11, feature11.size(2)).squeeze(2)
        # feature12 = F.max_pool1d(feature12, feature12.size(2)).squeeze(2)

        # feature1 = self.bn1(feature1)
        # feature2 = self.bn2(feature2)
        # feature3 = self.bn3(feature3)
        
        features = torch.cat((feature1, feature2, feature3), dim=1) #, feature4, feature5, feature6, feature7, feature8, feature9, feature10, feature11, feature12), dim=1)
        output = F.relu(features)
        output = self.FC1(output).squeeze(1)

        # output = self.dp(output)
        # output = self.FC2(output).squeeze(1)
        
        return output    

## General readmission case

In [9]:
vectors_all_genRe, labels_genRe = generate_dataset_oversample(df_data_genRe, pos_genRe_idx, neg_genRe_idx, 'GEN_RE')
x_train, x_test, y_train, y_test = train_test_split(vectors_all_genRe, labels_genRe, test_size=0.1, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)
dataset_train = HFDataset(x_train, y_train)
dataset_val = HFDataset(x_val, y_val)
dataset_test = HFDataset(x_test, y_test)

In [11]:
train_loader = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True, collate_fn=collate_data) # 128 is good and the default, decreasing batch size actually hurts performance (16 did not work, did not test others)
val_loader = DataLoader(dataset=dataset_val, batch_size=128, shuffle=True, collate_fn=collate_data)
test_loader = DataLoader(dataset=dataset_test, batch_size=128, shuffle=True, collate_fn=collate_data)

In [12]:
model = HFCNN(1024).to(device) # 1024 best so far, performance increases monotonically 
num_epoch = 50
sizes = {'training': len(x_train), 'validation': len(x_val)}
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0075) # 0.01 is the default, 0.0075 is the best so far, 0.02 is already too high and 0.005 is too low
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

best_val_loss = np.inf
# best_val_acc = 0.
train_loss_ls = []
val_loss_ls = []
train_acc_ls = []
val_acc_ls = []
for epoch in range(num_epoch):
    print('Epoch {} of {}'.format(epoch+1, num_epoch))
    print('-' * 20)
    for phase in ['training', 'validation']:
        if phase == 'training':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader
        epoch_loss = 0.
        epoch_correct = 0.
        for i, (input, target) in enumerate(dataloader):
            input = input.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'training'):
                output = model(input)
                loss = criterion(output, target)
                if phase == 'training':
                    loss.backward()
                    optimizer.step()
            epoch_loss += loss.item() * input.shape[0]
            preds = (F.sigmoid(output) > 0.5).float()
            epoch_correct += torch.sum(preds == target.data)

        epoch_loss = epoch_loss / sizes[phase]
        epoch_acc = (epoch_correct / sizes[phase]).item()
        
        if phase == 'training':
            train_loss_ls.append(epoch_loss)
            train_acc_ls.append(epoch_acc)
        else:
            val_loss_ls.append(epoch_loss)
            val_acc_ls.append(epoch_acc)
        
        # if phase == 'validation' and epoch_acc > best_val_acc:
        #     best_val_acc = epoch_acc
        #     torch.save(model.state_dict(), 'best_CNN_model.pth')
        
        print('{} phase, current loss is {}'.format(phase, epoch_loss))
        print('{} phase, current accuracy is {}'.format(phase, epoch_acc))
        print()
        
        if phase == 'validation' and epoch_loss < best_val_loss:
            best_val_loss = epoch_loss
            torch.save(model.state_dict(), 'best_CNN_model.pth')
            print('Best model saved')
            print()

training_curves = {'train loss': train_loss_ls, 'train accuracy': train_acc_ls, 'val loss': val_loss_ls, 'val accuracy': val_acc_ls}
training_curves = pd.DataFrame(training_curves)
training_curves.to_csv('training_curves.csv', index=False)

Epoch 1 of 50
--------------------
training phase, current loss is 0.6720027145491309
training phase, current accuracy is 0.5901699066162109

validation phase, current loss is 0.6361292267909376
validation phase, current accuracy is 0.5897714495658875

Best model saved

Epoch 2 of 50
--------------------
training phase, current loss is 0.6084549724280953
training phase, current accuracy is 0.6584244966506958

validation phase, current loss is 0.6079904202927183
validation phase, current accuracy is 0.638193666934967

Best model saved

Epoch 3 of 50
--------------------
training phase, current loss is 0.593874150305963
training phase, current accuracy is 0.6731153130531311

validation phase, current loss is 0.595323830940259
validation phase, current accuracy is 0.6697497367858887

Best model saved

Epoch 4 of 50
--------------------
training phase, current loss is 0.5821298356284239
training phase, current accuracy is 0.6844809651374817

validation phase, current loss is 0.607425466196

In [12]:
model = HFCNN(1024)
model.load_state_dict(torch.load('bestloss_CNN_model_1024out_00075WD.pth'))
model.to(device)
model.eval()
preds = []
targets = []
for i, (input, target) in enumerate(test_loader):
    input = input.to(device)
    targets += target.tolist()
    output = model(input)
    preds += (F.sigmoid(output) > 0.5).float().tolist()

acc = accuracy_score(targets, preds)
prec = precision_score(targets, preds)
rec = recall_score(targets, preds)
f1 = f1_score(targets, preds)

print('Test Accuracy is {:.4f}'.format(acc))
print('Test Precision is {:.4f}'.format(prec))
print('Test Recall is {:.4f}'.format(rec))
print('Test F1 score is {:.4f}'.format(f1))

Test Accuracy is 0.7499
Test Precision is 0.7512
Test Recall is 0.7519
Test F1 score is 0.7516


## 30 day readmission case

In [16]:
vectors_all_30Re, labels_30Re, neg_idx_sample = generate_dataset(df_data_30Re, pos_30Re_idx, neg_30Re_idx, '30_RE')
x_train, x_test, y_train, y_test = train_test_split(vectors_all_30Re, labels_30Re, test_size=0.1, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=True)

num_neg = len(x_train) // 2
neg_idx_unsampled = [i for i in neg_30Re_idx if i not in neg_idx_sample]
add_neg_idx = np.random.choice(neg_idx_sample, size=num_neg, replace=False).tolist()
add_x_neg = df_data.iloc[add_neg_idx, :]['TOKENS'].tolist()
add_x_neg = [vectorize(itm) for itm in add_x_neg]
add_y_neg = df_data.iloc[add_neg_idx, :]['30_RE'].tolist()
x_train = x_train + add_x_neg
y_train = y_train + add_y_neg
add_pos_idx = np.where(np.array(y_train) == 1)[0]
add_x_pos = [x_train[i] for i in add_pos_idx]
add_y_pos = [y_train[i] for i in add_pos_idx]
x_train = x_train + add_x_pos
y_train = y_train + add_y_pos

dataset_train = HFDataset(x_train, y_train)
dataset_val = HFDataset(x_val, y_val)
dataset_test = HFDataset(x_test, y_test)
print('{:.1f}% of training set are positive examples'.format(np.sum(y_train) / len(y_train) * 100))
print('{:.1f}% of validation set are positive examples'.format(np.sum(y_val) / len(y_val) * 100))
print('{:.1f}% of test set are positive examples'.format(np.sum(y_test) / len(y_test) * 100))

49.9% of training set are positive examples
49.4% of validation set are positive examples
51.8% of test set are positive examples


In [17]:
train_loader = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True, collate_fn=collate_data)
val_loader = DataLoader(dataset=dataset_val, batch_size=128, shuffle=True, collate_fn=collate_data)
test_loader = DataLoader(dataset=dataset_test, batch_size=128, shuffle=True, collate_fn=collate_data)

In [20]:
model = HFCNN(1024).to(device)
model.load_state_dict(torch.load('bestloss_CNN_model_30Re.pth'))
num_epoch = 5
sizes = {'training': len(x_train), 'validation': len(x_val)}
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.02)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

best_val_loss = np.inf
# best_val_acc = 0.
train_loss_ls = []
val_loss_ls = []
train_acc_ls = []
val_acc_ls = []
for epoch in range(num_epoch):
    print('Epoch {} of {}'.format(epoch+1, num_epoch))
    print('-' * 20)
    for phase in ['training', 'validation']:
        if phase == 'training':
            model.train()
            dataloader = train_loader
        else:
            model.eval()
            dataloader = val_loader
        epoch_loss = 0.
        epoch_correct = 0.
        for i, (input, target) in enumerate(dataloader):
            input = input.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'training'):
                output = model(input)
                loss = criterion(output, target)
                if phase == 'training':
                    loss.backward()
                    optimizer.step()
            epoch_loss += loss.item() * input.shape[0]
            preds = (F.sigmoid(output) > 0.5).float()
            epoch_correct += torch.sum(preds == target.data)

        epoch_loss = epoch_loss / sizes[phase]
        epoch_acc = (epoch_correct / sizes[phase]).item()
        
        if phase == 'training':
            train_loss_ls.append(epoch_loss)
            train_acc_ls.append(epoch_acc)
        else:
            val_loss_ls.append(epoch_loss)
            val_acc_ls.append(epoch_acc)
        
        # if phase == 'validation' and epoch_acc > best_val_acc:
        #     best_val_acc = epoch_acc
        #     torch.save(model.state_dict(), 'best_CNN_model.pth')
        
        print('{} phase, current loss is {}'.format(phase, epoch_loss))
        print('{} phase, current accuracy is {}'.format(phase, epoch_acc))
        print()
        
        if phase == 'validation' and epoch_loss < best_val_loss:
            best_val_loss = epoch_loss
            torch.save(model.state_dict(), 'best_CNN_model.pth')
            print('Best model saved')
            print()

training_curves = {'train loss': train_loss_ls, 'train accuracy': train_acc_ls, 'val loss': val_loss_ls, 'val accuracy': val_acc_ls}
training_curves = pd.DataFrame(training_curves)
training_curves.to_csv('training_curves.csv', index=False)

Epoch 1 of 5
--------------------
training phase, current loss is 0.588215542011812
training phase, current accuracy is 0.6815409660339355

validation phase, current loss is 0.6053535307960949
validation phase, current accuracy is 0.6954023241996765

Best model saved

Epoch 2 of 5
--------------------
training phase, current loss is 0.54387531691914
training phase, current accuracy is 0.7229534983634949

validation phase, current loss is 0.6020084441393271
validation phase, current accuracy is 0.6954023241996765

Best model saved

Epoch 3 of 5
--------------------
training phase, current loss is 0.5123018997056143
training phase, current accuracy is 0.7537720799446106

validation phase, current loss is 0.5957336603910074
validation phase, current accuracy is 0.7183908224105835

Best model saved

Epoch 4 of 5
--------------------
training phase, current loss is 0.4874409340836263
training phase, current accuracy is 0.7833065986633301

validation phase, current loss is 0.5932833237209539

In [21]:
model = HFCNN(1024)
model.load_state_dict(torch.load('best_CNN_model.pth'))
model.to(device)
model.eval()
preds = []
targets = []
for i, (input, target) in enumerate(test_loader):
    input = input.to(device)
    targets += target.tolist()
    output = model(input)
    preds += (F.sigmoid(output) > 0.5).float().tolist()

acc = accuracy_score(targets, preds)
prec = precision_score(targets, preds)
rec = recall_score(targets, preds)
f1 = f1_score(targets, preds)

print('Test Accuracy is {:.4f}'.format(acc))
print('Test Precision is {:.4f}'.format(prec))
print('Test Recall is {:.4f}'.format(rec))
print('Test F1 score is {:.4f}'.format(f1))

Test Accuracy is 0.7047
Test Precision is 0.6937
Test Recall is 0.7700
Test F1 score is 0.7299
